In [1]:
import os
import pandas as pd

In [3]:
from tqdm            import tqdm
from getpass         import getpass
from datasets        import Dataset, DatasetDict, Audio
from huggingface_hub import HfApi, HfFolder

In [4]:
from utils.config import load_config

In [5]:
config    = load_config()
audio_dir = config['data_paths']['wavefake']['audio_files']

### Hugging Face Token Setup


This notebook requires access to the Hugging Face API. You have several options to set up your environment with the necessary token.

**Option 1: Using Environment Variables**

1.a Setting Environment Variables Directly

Set the `HF_TOKEN` environment variable with your Hugging Face token. This can be done in your terminal:

**For Windows:**
```cmd
set HF_TOKEN=your_token_here
```

**For macOS/Linux:**
```bash
export HF_TOKEN=your_token_here
```

1.b Using a `.env` File

Create a `.env` file in the root directory of your project and add the following line:
```
HF_TOKEN=your_token_here
```

Then, use a library like `python-dotenv` to load the variables from the `.env` file. Install `python-dotenv` if you haven't already:
```bash
pip install python-dotenv
```

Add the following code to your script to load the environment variables:
```python
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv('HF_TOKEN')
```

**Option 2: Using Hugging Face CLI**

Run the following command in your terminal to log in using the Hugging Face CLI:
```bash
huggingface-cli login
```

**What It Does**

The `huggingface-cli login` command will prompt you to enter your Hugging Face credentials (username and password). After logging in, the CLI stores your token in a local file (usually located at `~/.huggingface/token`). This token is then used to authenticate your API requests without needing to re-enter your credentials.

**Example Code to Handle Token Setup in the Notebook**

Here’s an example of how to handle the Hugging Face token setup in your notebook, incorporating all the methods above:

```python
import os
from huggingface_hub import HfFolder, HfApi
from dotenv import load_dotenv

def get_hf_token():
    # Load environment variables from .env file, if it exists
    load_dotenv()
    
    # Check for the token in environment variables
    token = os.getenv('HF_TOKEN')
    if token is None:
        # If not found in environment variables, check the Hugging Face CLI session
        token = HfFolder.get_token()
    return token

# Attempt to retrieve the Hugging Face token
token = get_hf_token()

# If token is not set, prompt the user to enter it
if token is None:
    token = input("Please enter your Hugging Face token: ")

# Initialize Hugging Face API with the token
hf_api = HfApi(token)
```

### Uploading CodecFake Audio files to HuggingFace

In [6]:
# Attempt to retrieve the Hugging Face token from the environment variable or from the user's Hugging Face CLI session
token = HfFolder.get_token()

if token is None:
    token = getpass("Please enter your Hugging Face token: ")

# Initialize Hugging Face API with the token
hf_api = HfApi(token)

In [7]:
dir_list = ['ljspeech_melgan', 'ljspeech_full_band_melgan', 'ljspeech_hifiGAN', 'ljspeech_melgan_large', 'ljspeech_multi_band_melgan',
      'ljspeech_parallel_wavegan', 'ljspeech_waveglow', 'LJSpeech-1.1/wavs']

fake_label_mapping = {
    'ljspeech_melgan' : 'WF1',
    'ljspeech_full_band_melgan' : 'WF2',
    'ljspeech_hifiGAN' : 'WF3',
    'ljspeech_melgan_large' : 'WF4',
    'ljspeech_multi_band_melgan' : 'WF5',
    'ljspeech_parallel_wavegan' : 'WF6',
    'ljspeech_waveglow' : 'WF7',
    'LJSpeech-1.1/wavs' : 'R'
}

In [8]:
def create_df(audio_dir):
    data = []
    for dir in dir_list:
        print('processing dir:',dir)
        real_or_fake = fake_label_mapping[dir]
        print('real or fake label:', real_or_fake)
        if dir == "LJSpeech-1.1/wavs":
            cur_audio_dir = audio_dir+ '/LJSpeech-1.1/wavs'
        else:
            cur_audio_dir = audio_dir+ '/generated_audio/'+ dir
        print('audio dir:', cur_audio_dir)
        for root, _, files in tqdm(os.walk(cur_audio_dir), total=len(os.listdir(cur_audio_dir)), desc="Processing audio files"):
            for file in files:                
                file_path = os.path.join(root, file)
                cleaned_filename = os.path.basename(file_path)
                audio_id = cleaned_filename.replace('_generated','').replace('_gen','').replace('.wav', '')
                data.append({"audio": file_path, "audio_id": audio_id, "real_or_fake": real_or_fake})
        
    df = pd.DataFrame(data)
    return df

df = create_df(audio_dir)

processing dir: ljspeech_melgan
real or fake label: WF1
audio dir: /app/data/raw/wavefake/audio/generated_audio/ljspeech_melgan


Processing audio files:   0%|          | 1/13100 [00:00<46:31,  4.69it/s]


processing dir: ljspeech_full_band_melgan
real or fake label: WF2
audio dir: /app/data/raw/wavefake/audio/generated_audio/ljspeech_full_band_melgan


Processing audio files:   0%|          | 1/13100 [00:00<15:44, 13.88it/s]


processing dir: ljspeech_hifiGAN
real or fake label: WF3
audio dir: /app/data/raw/wavefake/audio/generated_audio/ljspeech_hifiGAN


...

Processing audio files:   0%|          | 1/13100 [00:00<17:54, 12.19it/s]


processing dir: ljspeech_waveglow
real or fake label: WF7
audio dir: /app/data/raw/wavefake/audio/generated_audio/ljspeech_waveglow


Processing audio files:   0%|          | 1/13100 [00:00<18:42, 11.67it/s]


processing dir: LJSpeech-1.1/wavs
real or fake label: R
audio dir: /app/data/raw/wavefake/audio/LJSpeech-1.1/wavs


Processing audio files:   0%|          | 1/13100 [00:00<16:46, 13.01it/s]


In [9]:
len(df)

104800

In [10]:
df.head(5)

,audio,audio_id,real_or_fake
0,/app/data/raw/wavefake/audio/generated_audio/l...,LJ013-0060,WF1
1,/app/data/raw/wavefake/audio/generated_audio/l...,LJ046-0060,WF1
2,/app/data/raw/wavefake/audio/generated_audio/l...,LJ022-0034,WF1
3,/app/data/raw/wavefake/audio/generated_audio/l...,LJ018-0179,WF1
4,/app/data/raw/wavefake/audio/generated_audio/l...,LJ021-0042,WF1


In [11]:
df[df['audio_id'] == 'LJ023-0093']

,audio,audio_id,real_or_fake
8231,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF1
21331,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF2
26200,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF3
47531,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF4
60631,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF5
73731,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF6
89080,/app/data/raw/wavefake/audio/generated_audio/l...,LJ023-0093,WF7
102180,/app/data/raw/wavefake/audio/LJSpeech-1.1/wavs...,LJ023-0093,R


In [12]:
group_counts = df.groupby('audio_id').size()

# Find audio_ids that do not have exactly 8 rows
invalid_audio_ids = group_counts[group_counts != 8].index.tolist()

# Print the audio_ids that do not have 8 rows
if invalid_audio_ids:
    print("Audio IDs with a row count different from 8:")
    for audio_id in invalid_audio_ids:
        print(audio_id)
else:
    print("All audio_ids have exactly 8 rows.")

All audio_ids have exactly 8 rows.


In [13]:
# Calculate the number of unique audio IDs
unique_audio_ids = df['audio_id'].nunique()
unique_audio_ids

13100

Since the file is large, we are breaking it into more manageable chunks (of desired chunk size between 90 and 110). 

To ensure that the file is divided evenly, we are checking for the best common divisor within this range.

In [14]:
desired_chunk_size_min = 90
desired_chunk_size_max = 110

def find_best_chunk_size(total_ids, min_size, max_size):
    for size in range(max_size, min_size - 1, -1):
        if total_ids % size == 0:
            return size
    raise ValueError("No suitable chunk size found within the given range.")

try:
    chunk_size = find_best_chunk_size(unique_audio_ids, desired_chunk_size_min, desired_chunk_size_max)
    num_chunks = unique_audio_ids // chunk_size
    print(f"Chunk Size: {chunk_size}, Number of Chunks: {num_chunks}")
except ValueError as e:
    print(e)

Chunk Size: 100, Number of Chunks: 131


In [15]:
chunk_index      = 0
datasets         = []
current_chunk    = []
audio_ids_so_far = set()

# Group the DataFrame by audio_id
grouped = df.copy().groupby('audio_id')

def read_audio_file(path):
    with open(path, 'rb') as f:
        return {'bytes': f.read()}

def save_chunk(chunk_data, chunk_index):
    chunk_name = f"partition{chunk_index}"
    df_chunk = pd.DataFrame(chunk_data)
    ds_chunk = Dataset.from_pandas(df_chunk)
    ds_chunk = ds_chunk.cast_column("audio", Audio(decode=True))
    datasets.append((chunk_name, ds_chunk))

# Iterate over the grouped data and create chunks
for audio_id, group in tqdm(grouped, desc="Creating chunks"):
    if audio_id in audio_ids_so_far:
        continue

    if len(audio_ids_so_far) + 1 > chunk_size:
        save_chunk(current_chunk, chunk_index)
        current_chunk = []
        audio_ids_so_far = set()
        chunk_index += 1

    current_chunk.extend(group.to_dict('records'))
    audio_ids_so_far.add(audio_id)

# Save the last chunk if it has remaining data
if current_chunk:
    save_chunk(current_chunk, chunk_index)

# Create a DatasetDict from the chunks
dataset_dict = DatasetDict({chunk_name: ds for chunk_name, ds in datasets})


Creating chunks: 100%|██████████| 13100/13100 [00:02<00:00, 5106.97it/s]


### Push it to HuggingFace Dataset Repository

In [16]:
repo_id = 'ajaykarthick/wavefake-audio'
dataset_dict.push_to_hub(repo_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ajaykarthick/wavefake-audio/commit/ca96a2a72cb3168da472c4ba5e543baa0740333e', commit_message='Upload dataset (part 00002-of-00003)', commit_description='', oid='ca96a2a72cb3168da472c4ba5e543baa0740333e', pr_url=None, pr_revision=None, pr_num=None)